# Grupo 20 - Tratamento

Filtragem dos dados

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree

zipcode = 98070

planilha = pd.read_excel("casas.xlsx")
planilha = planilha[planilha['zipcode'] == zipcode]
planilha_filtrada = planilha[['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','grade','sqft_above','sqft_basement','yr_built']]

planilha_filtrada.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built
10896,1350000.0,3,2.50,2753,65005,1.0,1,2,5,9,2165,588,1953
10897,369900.0,1,0.75,760,10079,1.0,1,4,5,5,760,0,1936
10898,309600.0,4,1.75,1275,20000,1.0,0,0,4,6,1275,0,1991
10899,517534.0,2,1.00,833,143947,1.0,0,0,3,5,833,0,2006
10900,705000.0,3,3.00,1970,20978,2.0,1,3,4,9,1770,200,1980


In [ ]:
#Usada apenas para encontrar os coeficientes

#scaler = MinMaxScaler()
#planilha_escala = pd.DataFrame(scaler.fit_transform(planilha_filtrada), columns=planilha_filtrada.columns)
#planilha_escala.to_excel("filtrada.xlsx")


Criação do critério de preço

In [18]:
coeficientes = 0.098,0,0.0165,0.1648,0.1882,0.0104,0.16,0.0209,0.0366,0.3256,0.0121,0.0309,0.0339


investimento = planilha_filtrada['price']
Base_medias = planilha_filtrada[['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','grade','sqft_above','sqft_basement','yr_built']].mean()

Base_indentada = planilha_filtrada / Base_medias
Base_indentada = Base_indentada * coeficientes
invest_medio = investimento.mean()

Col_razao = (investimento/invest_medio)/planilha_filtrada.sum(axis=1)


Categorização com base no critério

In [19]:
Base = planilha_filtrada
lista = []
media_Col_razao = Col_razao.mean()
desvio = 1.049
for i, row in Base.iterrows():
    if(Col_razao[i] <= media_Col_razao*1.08) and (Col_razao[i] >= media_Col_razao*0.93):
        lista.append('Medio')
    elif Col_razao[i] <= media_Col_razao:
        lista.append('Barato')
    else:
        lista.append('Caro')
Base.insert(13,"Caro_barato",lista,True)
Base.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,Caro_barato
10896,1350000.0,3,2.50,2753,65005,1.0,1,2,5,9,2165,588,1953,Caro
10897,369900.0,1,0.75,760,10079,1.0,1,4,5,5,760,0,1936,Caro
10898,309600.0,4,1.75,1275,20000,1.0,0,0,4,6,1275,0,1991,Caro
10899,517534.0,2,1.00,833,143947,1.0,0,0,3,5,833,0,2006,Barato
10900,705000.0,3,3.00,1970,20978,2.0,1,3,4,9,1770,200,1980,Caro


Seleção dos 10 ultimos registros para teste

In [10]:
Ultimos = Base.tail(10)
Base.drop(Base.tail(10).index,inplace=True)
Ultimos.head(10)

C:\Users\joao-\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,Caro_barato
11004,265000.0,0,0.75,384,213444,1.0,0,0,3,4,384,0,2003,Barato
11005,395000.0,4,2.75,1970,37026,1.0,0,0,4,8,1970,0,1961,Medio
11006,383000.0,3,1.50,1400,14850,1.5,0,0,4,7,1400,0,1910,Caro
11007,510000.0,3,1.75,2060,58341,1.0,0,4,3,8,1100,960,1982,Medio
11008,385000.0,2,1.00,830,26329,1.0,1,3,4,6,830,0,1928,Caro
11009,475000.0,4,3.50,3400,234352,2.0,0,0,3,8,2500,900,1991,Barato
11010,615000.0,2,1.75,2040,28593,1.5,1,3,4,7,2040,0,1919,Caro
11011,730000.0,2,1.75,1728,95950,1.0,0,3,3,9,1728,0,2012,Medio
11012,692000.0,3,2.50,3490,28213,1.5,0,2,3,9,2242,1248,2009,Caro
11013,420000.0,3,1.75,1444,249126,1.5,0,0,3,7,1444,0,2008,Barato


Criação da árvore

In [22]:
train = Base    
y_train = train['Caro_barato']
x_train = train.drop(['Caro_barato'], axis=1).values 
decision_tree = tree.DecisionTreeClassifier(max_depth = 20,criterion='gini',splitter='best')
decision_tree.fit(x_train, y_train)

with open("arvore.dot", 'w') as f:
     f = tree.export_graphviz(decision_tree,
                              out_file=f,
                              impurity = True,
                              feature_names = list(train.drop(['Caro_barato'], axis=1)),
                              #class_names = ['False', 'True'],
                              rounded = True,
                              filled= True )
        

In [23]:
Base.to_excel("base_treino.xlsx")

In [24]:
Ultimos.to_excel("base_teste.xlsx")